#### DataNatureで使うテキストファイル（タブ区切り）をDataFrameとして処理を行い、csvファイルとして出力する。
#### inとoutのファイル名を変更すること
#### 売上ファイル用

In [ ]:
# google drive mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# google driveのpathとinput、outputのファイル名を指定
input_path = "/content/drive/MyDrive/Purchased_products/txt/"
output_path = "/content/drive/MyDrive/Purchased_products/csv/"
input_file_name = "U2020_2022.txt"
output_file_name = "N_U_2020_2022.csv"

# 出力ファイルの項目を指定
columns_name = ""

In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
# テキストファイル読込
df = pd.read_table(input_path + input_file_name, encoding="cp932")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,55,57) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### 半角￥マークつきの金額部分の数値のみ取り出す
###### https://torisky.com/python%EF%BC%9A%E5%8D%8A%E8%A7%92%EF%BF%A5%E3%83%9E%E3%83%BC%E3%82%AF%E3%81%A4%E3%81%8D%E3%81%AE%E9%87%91%E9%A1%8D%E9%83%A8%E5%88%86%E3%81%AE%E6%95%B0%E5%80%A4%E3%81%AE%E3%81%BF%E5%8F%96%E3%82%8A/

In [ ]:
def mark_num_change(mark):
    """
        半角￥マークつきの金額部分の数値のみ取り出す
              例）販売価格 \824,529 --> 824529
    """
    if len(mark) >= 2:
        if mark[-2] == ".":
            mark = mark[:-2]
    # 「\」と「,」と数字以外の文字を消去する
    after = re.sub(r"\D", "", mark)
    # マイナス対応
    if "-" in mark:
        after = int(after) * (-1)
    else:
        after = int(after)
    return after

In [ ]:
# 列名を退避
columns = df.columns

In [ ]:
columns

In [ ]:
# nan対応
df = df.fillna(0)

In [ ]:
# DataFrameをnp.arrayに変換　処理スピードを上げるため
df = np.array(df)

In [ ]:
# M価、販売数量、単価、金額、予算金額、A価、仕切価格から￥などの文字を除いて数値にする
columns = list(columns)
a = columns.index("M価")
b = columns.index("販売数量")
c = columns.index("販売単価")
d = columns.index("販売金額")
e = columns.index("予算金額")
f = columns.index("A価")
# g = columns.index("仕切価格")

for i in range(len(df)):
    df[i, a] = mark_num_change(str(df[i, a]))
    df[i, b] = mark_num_change(str(df[i, b]))
    df[i, c] = mark_num_change(str(df[i, c]))
    df[i, d] = mark_num_change(str(df[i, d]))
    df[i, e] = mark_num_change(str(df[i, e]))
    df[i, f] = mark_num_change(str(df[i, f]))
    # df[i, g] = mark_num_change(str(df[i, g]))

In [ ]:
# 年度データ作成
h = columns.index("売上月")
nendo = np.array([[]])
for i in range(len(df)):
    if int(str(df[i, h])[-2:]) >= 6:
        nendo = np.append(nendo, int(str(df[i, h])[:4]))
    else:
        nendo = np.append(nendo, int(str(df[i, h])[:4]) - 1)

nendo = nendo.reshape(1, len(nendo))
df = np.append(df.T, nendo, axis=0).T

In [ ]:
# DataFrameに戻す
columns.append("年度")
df = pd.DataFrame(df, columns=columns)
df = df[df['請求先CD'] != 'Plastic Jaw Frame(UP-13)']
df = df[df['請求先CD'] != '114_']
df = df[df['請求先CD'] != ' ']

In [ ]:
# タイプをIntに
df = df[df['売上月']!=' ']
df = df.fillna(0)
df["M価"] = df["M価"].astype(int)

df['売上月'] = df['売上月'].astype(int)
df["販売数量"] = df["販売数量"].astype(int)
df["販売単価"] = df["販売単価"].astype(int)
df["販売金額"] = df["販売金額"].astype(int)
df["予算金額"] = df["予算金額"].astype(int)
df["A価"] = df["A価"].astype(int)
# df["仕切価格"] = df["仕切価格"].astype(int)
df["品目"] = df["品目"].astype(int)
df["請求先CD"] = df["請求先CD"].astype(int)

In [ ]:
df = df[df['伝票区分']!='予算']
df = df.reset_index(drop=True)

In [ ]:
col = ['請求先CD', '請求先名1', '請求先名2', '納入先CD', '納入先名', '学校', '年', '月', '日', '売上月', '売上日', '伝票区分',
       '売上区分', '部門', '得意先グループ', '品目', '品名', '規格', '大分類', '小分類', '販売数量', '販売単価', '販売金額', '年度']
df = df[col]

In [ ]:
# ファイル出力
df.to_csv(output_path + output_file_name, encoding="utf_8_sig", index=False)
# df.to_csv(output_path + output_file_name, encoding="cp932", index=False)